In this notebook we make predictions for all of the crowns which have treatment and geopmorphon information attached.  We will then attach the predictions to the crown df and save to `/TreeMortality/data/helena/predictions.parquet`.

In [2]:
import json
import os
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statistics import mode

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import rioxarray

from xrspatial import focal, slope
import seaborn as sns
from tqdm import tqdm
from joblib_progress import joblib_progress
from xrspatial.multispectral import ndvi, savi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss


In [3]:
# paths
helena_path = Path.cwd().parent / 'data' / 'helena'
feature_dir = helena_path / 'features'
crowns_path = helena_path / 'spectral_crowns' / 'crowns.parquet'

In [7]:
# years we will be looking at
years = [2018, 2020, 2022]

# get paths for features for years
parquets = {}
for y in years:
    parquets[y] = [feature_dir / p for p in os.listdir(feature_dir) if f'features_{y}_' in p]


parquets

{2018: [PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_1.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_2.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_4.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_10.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_8.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_3.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_6.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_11.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_7.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_5.parquet'),
  PosixPath('/home/michael/TreeMortality/data/helena/features/features_2018_9.parquet')],
 2020: [Posi

We will use the model which was tuned and trained in `src/mortality_classification.ipynb`.  It was pickled.

In [4]:
# load model created in src/mortality_classification.ipynb
pickle_path = Path.cwd() / 'RF_model.sav'
model = pickle.load(open(pickle_path, 'rb'))

Now we will make model predictions for the samples and create a timeseries of survival probabilities for each sample over the years for which we have NAIP data.

In [18]:
pred_list = []
for y in years:   
    
    # read parquet
    df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in parquets[y]
    )
    
    # make input features, X
    cols = list(model.feature_names_in_)
    X = df[cols]
    
    # make predictions
    pred = pd.DataFrame()
    pred['UniqueID'] = df['UniqueID']
    pred[f'pred_{y}'] = model.predict_proba(X)[:, 1]
    pred = pred.set_index('UniqueID')
    pred_list.append(pred)

-------features_2018_crowns_100.parquet----------
-------features_2020_crowns_100.parquet----------
-------features_2022_crowns_100.parquet----------


In [ ]:
# join all years into one df
pred_df = pd.concat(pred_list, axis=1)

# open desired columns of crowns
crowns = gpd.read_parquet(crowns_path)[[
    'UniqueID',
    'area',
    'zq95',
    'treatment',
    'geomorph_100',
    'geomorph_250',
    'geomorph_500',
    'geomorph_1000',
    'geomorph_2000',
    'geometry'
]].set_index('UniqueID')

# join crowns to predictions and move unique id back to a column
crowns = pd.concat([crowns, pred_df], axis=1).reset_index()

print(len(crowns), ' crowns.')
crowns.head()

In [36]:
crowns.to_parquet(helena_path / 'predictions.parquet')